In [2]:
import torch
import torchvision
from torchmetrics import Accuracy
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import random

from pathlib import Path
import zipfile
import requests
import os

from PIL import Image

from typing import Dict, List, Tuple

from torch.utils.tensorboard import SummaryWriter

> #### Defining the paths

In [4]:
import sys
import importlib.util

modules_path = 'PyTorch Bootcamp--06 -Transfer Learning/'
sys.path.append(modules_path)
module_names = ['data_setup', 'engine']
modules = {}
for module_name in module_names:
    spec = importlib.util.find_spec(module_name)
    if spec is not None:
        modules[module_name] = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(modules[module_name])
    else:
        print(f"Module '{module_name}' not found.")

In [8]:
from data_setup import create_dataloaders
from engine import train_step, test_step

In [6]:
data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'
image_paths_list = list(image_path.glob('*/*/*.jpg'))

train_path = image_path / 'train'
test_path = image_path / 'test'

> #### Automatic transformations

In [9]:
train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_path, test_path,
    transform=torchvision.models.EfficientNet_B0_Weights.DEFAULT.transforms(),
    batch_size=32)

> ### Tensorboard

In [14]:
writer = SummaryWriter()

In [15]:
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int) -> Dict[str, List]:
    
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
              }
    
    for epoch in range(epochs):
        
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn)
        
        print(f"Epoch: {epoch+1} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} |"
          f"test_loss: {test_loss:.4f} | " f"test_acc: {test_acc:.4f}")
        
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        writer.add_scalars(main_tag='Loss',
                          tag_scalar_dict={"train_loss": train_loss,
                                           "test_loss": test_loss},
                          global_step=epoch)
        
        writer.add_scalars(main_tag='Accuracy',
                          tag_scalar_dict={"train_acc": train_acc,
                                           "test_acc": test_acc},
                          global_step=epoch)
        
        writer.add_graph(model=model,
                        input_to_model=torch.rand(32, 3, 224, 224))
        
    writer.close()
        
    return results

In [29]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

eff_net_model = torchvision.models.efficientnet_b0(weights=weights)

for param in eff_net_model.features.parameters():
    param.requires_grad = False

eff_net_model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=3, bias=True))

In [30]:
EPOCHS = 4
BATCH_SIZE = 32
learning_rate = 0.001

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(eff_net_model.parameters(), lr=learning_rate)

In [31]:
model_results = train(epochs=EPOCHS, model=eff_net_model,
                      train_dataloader=train_dataloader, test_dataloader=test_dataloader,
                      loss_fn=loss_fn, optimizer=optimizer)

Epoch: 1 | train_loss: 1.0648 | train_acc: 0.4688 |test_loss: 0.8456 | test_acc: 0.7453
Epoch: 2 | train_loss: 0.8869 | train_acc: 0.6484 |test_loss: 0.7051 | test_acc: 0.8561
Epoch: 3 | train_loss: 0.7512 | train_acc: 0.7539 |test_loss: 0.6952 | test_acc: 0.8258
Epoch: 4 | train_loss: 0.6917 | train_acc: 0.7461 |test_loss: 0.6304 | test_acc: 0.8665


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs